RANSAC with SIFT descriptor

In [3]:
import cv2
import numpy as np
from skimage.measure import ransac as _ransac
from skimage.transform import AffineTransform
import matplotlib.pyplot as plt


def ransac(img1, img2, kp1, kp2, des1, kes2, is_binary_descriptor, match_method, description):
    # find the keypoints and descriptors
#     kp1 = detector.detect(img1,None)
#     kp1, des1 = descriptor_extractor.compute(img1, kp1)

#     kp2 = detector.detect(img2,None)
#     kp2, des2 = descriptor_extractor.compute(img2, kp2)

    matches = get_matches(des1, des2, 'BRUTE_FORCE', is_binary_descriptor=is_binary_descriptor)
    matches = matches[:100] # only shows top 100 matches

    # # Draw matches. flag: cv2.DrawMatchesFlags.DRAW_RICH_KEYPOINTS 4
    # matching_img = cv2.drawMatches(img1, kp1, img2, kp2, matches, None, flags=0)
    # matching_img = cv2.cvtColor(matching_img, cv2.COLOR_BGR2RGB);

    # plt.figure(figsize=(16, 12))
    # plt.imshow(matching_img)
    # plt.show()


    locations_1_to_use = []
    locations_2_to_use = []

    for match in matches:
        locations_1_to_use.append((kp1[match.queryIdx].pt[1], kp1[match.queryIdx].pt[0])) # (row, col)
        locations_2_to_use.append((kp2[match.trainIdx].pt[1], kp2[match.trainIdx].pt[0]))

    locations_1_to_use = np.array(locations_1_to_use)
    locations_2_to_use = np.array(locations_2_to_use)

    # Perform geometric verification using RANSAC.
    model_robust, inliers = _ransac(
      (locations_1_to_use, locations_2_to_use),
      AffineTransform,
      min_samples=5,
      residual_threshold=10,
      max_trials=1000)    

    inlier_idxs = np.nonzero(inliers)[0]

    inlier_match = []
    for idx in inlier_idxs:
        inlier_match.append(matches[idx])

    # Does ransac consider size and orientation of keypoints?
    ransac_img = cv2.drawMatches(img1, kp1, img2, kp2, inlier_match, None, flags=0)    

    if inliers is None:
        score = 0
    else:
        score = len(inliers)

    return ransac_img, score


def draw_ransac(img1, img2, kp1, kp2, des1, kes2, is_binary_descriptor, match_method, description):
    ransac_img, score = ransac(img1, img2, detector, descriptor_extractor, is_binary_descriptor, match_method, description)
    ransac_img = cv2.cvtColor(ransac_img, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(16, 12))
    plt.title(description)
    plt.imshow(ransac_img)
    plt.show()
    return score

    

In [ ]:

def parse_sift_output(target_path="a.hessian.sift"):
    """
    Return:
        kp: keypoint of hessian affine descriptor. location, orientation etc... OpenCV KeyPoint format. 
        des: 128d uint8 np array
    """
    
    return kp, des
    
